<p align="center">
    <br clear="right"/>
    <img src="https://ychef.files.bbci.co.uk/1600x900/p086c892.webp" alt="Books" width="1000" height="300" />
</p>

In [43]:
''' Load the neccessary Library to Run the website and to do the task '''

from selenium import webdriver           
import requests                          
from bs4 import BeautifulSoup             
import pandas as pd                      
import time                              


In [80]:
def login():
    '''
    This function loads the web browser driver which is Chrome from the path 
    where it is installed on the computer and opens it. This driver works well
    with the selenium library.

    '''
    
    driver = webdriver.Chrome(executable_path="C:\chromedriver.exe")
    return driver

# driver = login()

In [82]:
# select_abuja= driver.find_element_by_link_text(state.title())
# select_abuja.click()

In [101]:
#State selection to consider 

pick_any = ['abia','abuja','adamawa','akwa ibom','anambra','bauchi','bayelsa','benue',
'borno','cross river','delta','ebonyi','edo','ekiti','enugu','gombe','imo',
'jigawa','kaduna','kano','katsina','kebbi','kogi','kwara','lagos', 
'nasarawa','niger','ogun','ondo','osun','oyo','plateau','rivers','sokoto',
'taraba','yobe','zamfara']

def userInput():
    ''' 
    This function accepts user input of states to look for, then if the state is present in the database,
    it gives the state. while it gives error when it is not a part of the database.

    '''  
    # ASk for the user choice among the 36 states of Nigeria and the FCT
    user_input = input("What state do you want to consider?: ").casefold()           
    
    while user_input not in pick_any:

        print("Hey user, that is a wrong entry why not put space between or add another state")
        
     # If the user input is not part of the 36 states, Ask for another entry
        user_input = input("What state do you want to consider?: ").casefold()
    print('you entered:', user_input)

    return user_input

#entry = userInput()
        

In [85]:
def exempt(state: str):
    
    '''
    Since Cross river state and Akwa Ibom is two word unlike the other states,
    give the reponse in another format based on the website code structure.  

    '''
    # if the user entry is gombe, then there is no records of property for the state
    if state == pick_any[15]:
        print("No records available for this state")
    
    # if the entry is akwa ibom or cross river, return in the format understood by the website code architecture     
    elif state == pick_any[3]:
        state = pick_any[3].replace(pick_any[3],'akwa-ibom')
        return state

    elif state == pick_any[9]:
        state = pick_any[9].replace(pick_any[9],'cross-river')
        return state
    
    else:
        return state

In [102]:
storage = [] 

def get_info(driver,state_info, start: int, stop: int, step: int):
    
    '''   
    Get the website using the driver and selinium library then find the data 
    available in the elements paths, class or selector of the website raw code 
    file.
    Then extract the property type, location of property, price of property and
    other feature of the property.
    This function iterates to give result of times the user gives.
    '''


    while True:

        try:
            for page in range(start,stop,step):
                website = 'https://www.propertypro.ng/in/'+ state_info +'?page='+str(page) 
                driver.get(website)
                
                property_type = driver.find_elements_by_css_selector(".listings-property-title")
                location = driver.find_elements_by_xpath('//div[2]/h4')
                price = driver.find_elements_by_xpath("//h3//span[2]")
                features = driver.find_elements_by_class_name('fur-areea')
                for i in range(len(property_type)):
                    storage.append([property_type[i].text, location[i].text, price[i].text, features[i].text])
                    time.sleep(1)
                    
        except:
            print("Not Available")
            break
        return storage

In [103]:
def statistics_to_pandas(site_data: list):
    '''  
    Load the data generated from the scraping into a data frame and clean the data to desired taste/result
    
    '''
    df = pd.DataFrame(site_data, columns=["Type of House","Location", "Price", "Features"])  # Create a dataframe using the scrapped data  

    df['Purpose'] = df['Type of House'].str.split(" ").map(lambda x: x[-1])                  # Determine the what the property owner wants either to sell or Rent the property
    df['Type of House'] = df['Type of House'].str.split("FOR").map(lambda x: x[0])           # Get the type of property being considered
    
    print("="*40)
    return df


In [104]:
def main():

    while True:
        driver = login()
        
        entry = userInput()

        # I am going to search for property in Abuja
        state_info = exempt(entry)

        # I would be searching for the first 3 pages or less
        site_data = get_info(driver,state_info,0,3,1)
        
        # Give the data in Data Frame for better output
        statistics_to_pandas(site_data) 

        restart = input('\nWould you like to restart? Enter yes or no.\n')
        if restart.lower() != 'yes':
            break
    return statistics_to_pandas(site_data)       

main()
    

you entered: abuja


,Type of House,Location,Price,Features,Purpose
0,4 BEDROOM HOUSE,"Mekunwen Road, Brains And Hammers Estate, Life...","85,000/DAY",4 beds 4 baths 5 Toilets,SHORTLET
1,4 BEDROOM HOUSE,Just By Maitama Katampe Main Abuja,"95,000,000",4 beds 5 baths 5 Toilets,SALE
2,4 BEDROOM HOUSE,Apo Resettlement Apo Abuja,"190,000,000",4 beds 5 baths 5 Toilets,SALE
3,5 BEDROOM HOUSE,Mabushi Abuja,"180,000,000",5 beds 6 baths 6 Toilets,SALE
4,4 BEDROOM HOUSE,Jahi Abuja,"140,000,000",4 beds 5 baths 5 Toilets,SALE
...,...,...,...,...,...
61,4 BEDROOM HOUSE,Side Of Coza Guzape Abuja,"120,000,000",4 beds 4 baths 5 Toilets,SALE
62,LAND,"Hilton City, Kurudu Abuja","4,800,000",0 beds 0 baths 0 Toilets,SALE
63,LAND,"Close To Mtn Office, Maitama Abuja","1,500,000,000",0 beds 0 baths 0 Toilets,SALE
64,4 BEDROOM HOUSE,"Nearby: Brains & Hammer Estate, Galadinmawa Abuja","65,000,000",4 beds 5 baths 5 Toilets,SALE


In [99]:
#driver.quit()    #Quit the browser at the end of the scraping.